# ARPS Headquarters: A Real State study

## Table of Contents

1. <a href="#Introduction">Introduction. Business Problem</a>  
2. <a href="#Data-description">Data description</a>  
3. <a href="#Data-analysis">Data analysis</a>  
4. <a href="#Clustering-Neighborhoods">Clustering Neighborhoods</a>  

## Introduction

The ARPS (A Really Promising Startup) company, very committed to technological development, ecological culture, and the well-being of its workers, has grown very rapidly in the last year thanks to its good work and its innovative business model. For this reason, it anticipates going from having 13 workers to 50 in the next year, and has commissioned me to carry out a study in order to choose the stablishment for the new headquarters in the city of Madrid.

This is a great investment for them, but as they have always done, they will know how to make a great profit by previously doing a good study.

My job will be to show which are the best areas to buy an office in Madrid. The characteristics to be valued are:

- Have a permissive price
- That the trend of real estate revaluation of the area is high
- That it is well connected, with metro and train stations nearby
- Have nurseries nearby


## Data description

1. To build the model, I will use public data from the Madrid City Council regarding the price of second-hand real estate by neighborhoods and districts, which can be found at the following URL:  
http://www-2.munimadrid.es/CSE6/control/seleccionDatos?numSerie=05040300200
2. To search for metro and train stations, as well as nurseries in each neighborhood, I will use the Foursquare API:  
https://developer.foursquare.com/docs/api
3. The coordinates of each neighborhood will be obtained using the OpenStreetMaps.org Nominatim API:  
https://nominatim.org/release-docs/develop/api/Overview/
4. In order to segment and visualize each neighborhood area, I will use the cartodb GeoJSON from:  
https://ayuntamiento-madrid.carto.com/u/ayuntamientomadrid/tables/cartodb_query/public?


Once the data is collected and cleaned, I will compute the score of each neighborhood, considering the characteristics to be valued described in the Problem description section.
This will be done using k-means clustering, grouping neighborhood candidates in a scaled range from worst candidates to best candidates clusters

In [1]:
!conda install -c conda-forge geopy folium shapely  --yes 
!conda install -c anaconda unidecode django --yes

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import unidecode
import os
from time import sleep
import re
import json
import folium
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from folium import plugins
from folium.plugins import HeatMap
from shapely.geometry import shape, Point, Polygon

Solving environment: ...working... 
  - anaconda::ca-certificates-2019.8.28-0, anaconda::certifi-2019.9.11-py37_0, anaconda::openssl-1.1.1d-he774522_2
  - anaconda::ca-certificates-2019.8.28-0, anaconda::openssl-1.1.1d-he774522_2, defaults::certifi-2019.9.11-py37_0
  - anaconda::certifi-2019.9.11-py37_0, anaconda::openssl-1.1.1d-he774522_2, defaults::ca-certificates-2019.8.28-0
  - anaconda::openssl-1.1.1d-he774522_2, defaults::ca-certificates-2019.8.28-0, defaults::certifi-2019.9.11-py37_0
  - anaconda::certifi-2019.9.11-py37_0, defaults::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-he774522_2
  - defaults::ca-certificates-2019.8.28-0, defaults::certifi-2019.9.11-py37_0, defaults::openssl-1.1.1d-he774522_2
  - anaconda::ca-certificates-2019.8.28-0, anaconda::certifi-2019.9.11-py37_0, defaults::openssl-1.1.1d-he774522_2
  - anaconda::ca-certificates-2019.8.28-0, defaults::certifi-2019.9.11-py37_0, defaults::openssl-1.1.1d-he774522_2done

## Package Plan ##

  environment locat

## Data analysis

Data from Madrid City Council has been downloaded to prices.csv. Let's explore its contents

In [2]:
precios_df = pd.read_csv('prices.csv', sep=';')

In [3]:
precios_df.head()

,District,Neighborhood,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,01. Centro,01. Centro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01. Centro,01. Centro,2.305,2.983,3.418,3.870,4.289,4.603,4.728,4.376,4.161,4.049,3.823,3.424,3.354,3.285,3.568,3.779,4.599,4.975
2,01. Centro,011. Palacio,2.610,3.279,3.605,4.128,4.503,4.555,4.807,4.544,4.239,4.113,3.850,3.519,3.397,3.369,3.682,3.731,4.474,4.873
3,01. Centro,012. Embajadores,2.107,2.715,3.022,3.401,3.852,4.252,4.394,3.782,3.601,3.489,3.215,2.721,2.897,2.697,3.000,3.240,4.017,4.239
4,01. Centro,013. Cortes,2.611,3.070,3.683,4.107,4.610,4.858,4.922,4.401,4.101,4.272,4.043,3.715,3.358,3.541,3.714,3.616,4.842,5.106


In [4]:
precios_df.shape

(181, 20)

Not all the rows are Neighborhoods, lets clean it, removing rows with average District data. Also, rows with no data in all years columns will be droped

In [5]:
precios_df.dropna(how='all', subset=precios_df.columns[range(2, precios_df.shape[1])], inplace=True)

In [6]:
precios_df['District']=precios_df['District'].map(lambda x: unidecode.unidecode(x))
precios_df['Neighborhood']=precios_df['Neighborhood'].map(lambda x: unidecode.unidecode(x))

In [7]:
precios_df['District']=precios_df['District'].map(lambda x: x.lstrip("0123456789 ."))
precios_df['Neighborhood']=precios_df['Neighborhood'].map(lambda x: x.lstrip("0123456789 ."))

In [8]:
for i in range(2,precios_df.shape[1]):
    precios_df.iloc[:,i]=pd.to_numeric(precios_df.iloc[:,i].map(lambda s: s.replace('.', '')), errors='coerce')

precios_df.dropna(how='all', subset=precios_df.columns[range(2, precios_df.shape[1])], inplace=True)

In [9]:
precios_df.shape

(158, 20)

In [10]:
precios_df.drop(precios_df.index[(precios_df.District == precios_df.Neighborhood)], inplace=True)
precios_df.reset_index(drop=True, inplace=True)

Exploring again the resulting data, it has been found that the table contains subareas in some Neighborhoods. They are not necessary and we can compute the mean values in this cases, and assign them to the Neighborhood:

In [11]:
precios_df['Neighborhood']=precios_df['Neighborhood'].map(lambda x: re.sub(' *\(.*\) *', '', x))
precios_df= precios_df.groupby(['District','Neighborhood']).mean().reset_index()

Now it's time to find the coordinates of each Neighborhood center using Nominatim API:

In [12]:
geolocator = Nominatim(user_agent="my-capstone-project2634")
from geopy.exc import GeocoderTimedOut

def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [13]:
precios_df['coordinates'] = (precios_df['Neighborhood']+", "+precios_df['District']+", Madrid, Spain").apply(do_geocode).apply(lambda x: (x.latitude, x.longitude) if x != None else '-')

The resulting DataFrame, with the coordinates appended is:

In [14]:
precios_df

,District,Neighborhood,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,coordinates
0,Arganzuela,Acacias,2237.0,2993.0,3506.0,3793.0,4178.0,4453.0,4309.0,3940.0,...,3588.0,3365.0,2957.0,2918.0,2804.0,2966.0,3193.0,3724.0,4046.0,"(40.4040749, -3.7059572)"
1,Arganzuela,Atocha,2312.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(40.40068665, -3.68239115386921)"
2,Arganzuela,Chopera,1939.0,2576.0,3222.0,3434.0,3680.0,3972.0,3942.0,3503.0,...,3123.0,2894.0,2379.0,2267.0,2190.0,2355.0,2649.0,3340.0,3727.0,"(40.39489315, -3.69970511346301)"
3,Arganzuela,Delicias,2254.0,3008.0,3279.0,3751.0,3882.0,4295.0,4138.0,3820.0,...,3375.0,3206.0,2825.0,2710.0,2564.0,2592.0,3025.0,3566.0,3777.0,"(40.39729215, -3.68949484969473)"
4,Arganzuela,Imperial,2264.0,2934.0,3591.0,3507.0,3711.0,4254.0,4240.0,3723.0,...,3352.0,3104.0,2816.0,2752.0,2731.0,2697.0,2920.0,3382.0,3896.0,"(40.4069151, -3.71732906254996)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Villaverde,Angeles,1476.0,1959.0,2314.0,2782.0,2908.0,3095.0,2992.0,2566.0,...,2203.0,1835.0,1532.0,1457.0,1271.0,1357.0,1404.0,1551.0,1648.0,"(40.3551089, -3.70010471415375)"
124,Villaverde,Butarque,1347.0,1547.0,NaN,2789.0,2779.0,3039.0,3065.0,2656.0,...,2377.0,2178.0,1832.0,1784.0,1515.0,1594.0,1772.0,1840.0,1803.0,"(40.3399339, -3.67338785851676)"
125,Villaverde,Rosales,1450.0,2060.0,2310.0,2732.0,2975.0,3071.0,3064.0,2753.0,...,2305.0,1951.0,1707.0,1492.0,1322.0,1302.0,1500.0,1438.0,1761.0,"(40.3558406, -3.68839888963864)"
126,Villaverde,San Andres,1468.0,1922.0,2239.0,2685.0,2800.0,3153.0,2985.0,2587.0,...,2108.0,1797.0,1563.0,1321.0,1193.0,1195.0,1291.0,1390.0,1644.0,"(40.3455389, -3.7109697)"


In [15]:
precios_df[['Latitude', 'Longitude']] = precios_df['coordinates'].apply(pd.Series)
precios_df.dropna(how='any', subset=["Latitude","Longitude"], inplace=True)

In [16]:
precios_df.drop(columns=['coordinates'], inplace=True)

In cases with any year column has no price, we will fill it with the maximum value found in the 10 years.

In [17]:
precios_df[precios_df.columns[range(2, precios_df.shape[1]-2)]]= precios_df[precios_df.columns[range(2, precios_df.shape[1]-2)]].apply(lambda x: x.fillna(x.max()),axis=1)
precios_df

,District,Neighborhood,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,Latitude,Longitude
0,Arganzuela,Acacias,2237.0,2993.0,3506.0,3793.0,4178.0,4453.0,4309.0,3940.0,...,3365.0,2957.0,2918.0,2804.0,2966.0,3193.0,3724.0,4046.0,40.4041,-3.705957
1,Arganzuela,Atocha,2312.0,2312.0,2312.0,2312.0,2312.0,2312.0,2312.0,2312.0,...,2312.0,2312.0,2312.0,2312.0,2312.0,2312.0,2312.0,2312.0,40.4007,-3.682391
2,Arganzuela,Chopera,1939.0,2576.0,3222.0,3434.0,3680.0,3972.0,3942.0,3503.0,...,2894.0,2379.0,2267.0,2190.0,2355.0,2649.0,3340.0,3727.0,40.3949,-3.699705
3,Arganzuela,Delicias,2254.0,3008.0,3279.0,3751.0,3882.0,4295.0,4138.0,3820.0,...,3206.0,2825.0,2710.0,2564.0,2592.0,3025.0,3566.0,3777.0,40.3973,-3.689495
4,Arganzuela,Imperial,2264.0,2934.0,3591.0,3507.0,3711.0,4254.0,4240.0,3723.0,...,3104.0,2816.0,2752.0,2731.0,2697.0,2920.0,3382.0,3896.0,40.4069,-3.717329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Villaverde,Angeles,1476.0,1959.0,2314.0,2782.0,2908.0,3095.0,2992.0,2566.0,...,1835.0,1532.0,1457.0,1271.0,1357.0,1404.0,1551.0,1648.0,40.3551,-3.700105
124,Villaverde,Butarque,1347.0,1547.0,3065.0,2789.0,2779.0,3039.0,3065.0,2656.0,...,2178.0,1832.0,1784.0,1515.0,1594.0,1772.0,1840.0,1803.0,40.3399,-3.673388
125,Villaverde,Rosales,1450.0,2060.0,2310.0,2732.0,2975.0,3071.0,3064.0,2753.0,...,1951.0,1707.0,1492.0,1322.0,1302.0,1500.0,1438.0,1761.0,40.3558,-3.688399
126,Villaverde,San Andres,1468.0,1922.0,2239.0,2685.0,2800.0,3153.0,2985.0,2587.0,...,1797.0,1563.0,1321.0,1193.0,1195.0,1291.0,1390.0,1644.0,40.3455,-3.710970


A new column with the mean revaluation of each Neighborhood is appended to our data. It is relevant for our client

In [18]:
precios_df["pct_inc_year"]=precios_df[precios_df.columns[range(2, precios_df.shape[1]-2)]].pct_change(axis='columns').mean(axis='columns')*100

In [19]:
precios_df.reset_index(inplace=True, drop=True)

Now lets prepare a Dataframe to show the prices evolution in a HeatMap

In [20]:
df_mapa_precios=precios_df.drop(columns=precios_df.columns[range(0, 2)]).reset_index(drop=True)
df_mapa_precios=df_mapa_precios.drop(columns=df_mapa_precios.columns[df_mapa_precios.shape[1]-1]).reset_index(drop=True)

In [21]:
cols = df_mapa_precios.columns.tolist()
cols = cols[-2:] + cols[:-2]
df_mapa_precios = df_mapa_precios[cols]

We need to normalize the data. It will be done with a minmal scale to fin between [0,1)

In [22]:
from sklearn import preprocessing
for index in range(2,df_mapa_precios.shape[1]):
    x = df_mapa_precios.iloc[:,index].values.reshape(-1, 1)  #returns a numpy array
    x_scaled = preprocessing.minmax_scale(x)
    df_mapa_precios.iloc[:,index] = x_scaled
df_mapa_precios

,Latitude,Longitude,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,40.4041,-3.705957,0.353513,0.467507,0.400289,0.458372,0.498531,0.455241,0.418483,0.368388,0.381535,0.372697,0.406580,0.428800,0.382336,0.365675,0.415006,0.430639,0.446016,0.418326
1,40.4007,-3.682391,0.377039,0.247333,0.055700,0.000000,0.000000,0.000000,0.000000,0.029363,0.063181,0.119873,0.188703,0.281136,0.264184,0.270584,0.282025,0.260660,0.211113,0.158706
2,40.3949,-3.699705,0.260038,0.332687,0.318326,0.347261,0.365482,0.352966,0.341576,0.277384,0.286541,0.280563,0.309125,0.296474,0.255410,0.247004,0.290769,0.325680,0.382133,0.370564
3,40.3973,-3.689495,0.358846,0.472357,0.334776,0.445373,0.419450,0.421646,0.382649,0.343399,0.326808,0.330493,0.373681,0.398581,0.341782,0.319289,0.338959,0.398225,0.419730,0.378051
4,40.4069,-3.717329,0.361982,0.448432,0.424820,0.369855,0.373764,0.412928,0.404023,0.323199,0.336596,0.325936,0.352576,0.396520,0.349971,0.351566,0.360309,0.377966,0.389120,0.395868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,40.3551,-3.700105,0.114806,0.133204,0.056277,0.145466,0.159231,0.166489,0.142498,0.082257,0.090323,0.098276,0.090006,0.102564,0.097485,0.069385,0.087841,0.085472,0.084512,0.059290
119,40.3399,-3.673388,0.074341,0.000000,0.273016,0.147632,0.124766,0.154582,0.157795,0.101000,0.118131,0.132752,0.160977,0.171245,0.161240,0.116544,0.136031,0.156473,0.132590,0.082497
120,40.3558,-3.688399,0.106650,0.165858,0.055123,0.129991,0.177131,0.161386,0.157586,0.121200,0.117909,0.118486,0.114008,0.142628,0.104309,0.079242,0.076657,0.103994,0.065713,0.076209
121,40.3455,-3.710970,0.112296,0.121242,0.034632,0.115444,0.130377,0.178822,0.141031,0.086631,0.071858,0.079453,0.082144,0.109661,0.070969,0.054310,0.054900,0.063670,0.057727,0.058691


Now let's load the GeoJson to segment each Neighborhood area in the city of Madrid and visualize with folium

In [23]:
madrid_center_coords=(40.4167047,-3.7035825)

map_madrid = folium.Map(location=madrid_center_coords, zoom_start=11)

with open('madrid.geojson','r', encoding='utf-8') as f:
    madrid_neighborhoods = json.load(f)
    folium.GeoJson(madrid_neighborhoods, name='geojson').add_to(map_madrid)

Also, we will add a circle marker in the center of each Neighborhood to show relevant data.

In [24]:
for lat, lng, District, Neighborhood in zip(precios_df['Latitude'], precios_df['Longitude'], precios_df['District'], precios_df['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, District)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(map_madrid)

In case any Neighborhood has changed between the GeoJson and our Table from the City Council, we can also compute the center of each Polygon inside the GeoJson. It is done the following way:

In [25]:
for feature in madrid_neighborhoods['features']:
    feature['centroid']=np.average(np.squeeze(feature['geometry']['coordinates']), axis=0)[::-1].tolist()

Now let's plot this new centers from the GeoJson polygons with red color dots:

In [26]:
for feature in madrid_neighborhoods['features']:
    label = unidecode.unidecode(feature['properties']['name'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        feature['centroid'],
        radius=2,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(map_madrid)

In [27]:
HeatMap(df_mapa_precios.iloc[:,[0,1,19]]).add_to(map_madrid)
map_madrid

It is not necessary to use this new centers, let's keep the original ones from Nominatim API.

Now, a HeatMap with Time is drown to show the prices evolution from 2001 to 2018.

In [28]:
from folium.plugins import HeatMapWithTime
map_madrid2 = folium.Map(location=madrid_center_coords, zoom_start=11)
folium.GeoJson(madrid_neighborhoods, name='geojson').add_to(map_madrid2)
auxlist=[]
for index in range(2,df_mapa_precios.shape[1]):
    auxlist.append(df_mapa_precios.iloc[:,[0,1,index]].values.tolist())
HeatMapWithTime(auxlist).add_to(map_madrid2)
map_madrid2.save (r'prices_heatmapwithtime.html')
map_madrid2

In [29]:
precios_df.fillna(0,inplace=True)

A good idea can be to show also the evolution of the revaluation of these areas. Let's compute a new table with the normalized price increase per year

In [30]:
df_mapa_inc_years=pd.concat([precios_df['District'],precios_df['Neighborhood'], precios_df['Latitude'], precios_df['Longitude']], axis=1, keys=['District','Neighborhood','Latitude', 'Longitude'])
for i in range(5, precios_df.shape[1]-3):
    df_mapa_inc_years["inc_year_"+precios_df.columns[i]]=precios_df[precios_df.columns[range(i-1, i+1)]].pct_change(axis='columns').mean(axis='columns')

for i in range(4,df_mapa_inc_years.shape[1]):
    x = df_mapa_inc_years.iloc[:,i].values.reshape(-1, 1)  #returns a numpy array
    x_scaled = (preprocessing.maxabs_scale(x)+1)/2
    df_mapa_inc_years.iloc[:,i] = x_scaled
df_mapa_inc_years

,District,Neighborhood,Latitude,Longitude,inc_year_2004,inc_year_2005,inc_year_2006,inc_year_2007,inc_year_2008,inc_year_2009,inc_year_2010,inc_year_2011,inc_year_2012,inc_year_2013,inc_year_2014,inc_year_2015,inc_year_2016,inc_year_2017,inc_year_2018
0,Arganzuela,Acacias,40.404075,-3.705957,0.562082,0.709603,0.663632,0.324509,0.308452,0.406599,0.444097,0.422095,0.387514,0.493044,0.467920,0.518554,0.561445,0.550289,0.547569
1,Arganzuela,Atocha,40.400687,-3.682391,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
2,Arganzuela,Chopera,40.394893,-3.699705,0.549901,0.647929,0.697260,0.459012,0.250899,0.400280,0.421429,0.408087,0.334906,0.475171,0.472109,0.524196,0.600227,0.578882,0.563744
3,Arganzuela,Delicias,40.397292,-3.689495,0.609168,0.572118,0.764484,0.301627,0.328105,0.342050,0.452904,0.437234,0.389749,0.478531,0.455761,0.503507,0.634116,0.554082,0.532552
4,Arganzuela,Imperial,40.406915,-3.717329,0.482260,0.620119,0.863759,0.482140,0.227258,0.408681,0.427946,0.407262,0.413922,0.488014,0.493734,0.496002,0.566382,0.547845,0.583611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,Villaverde,Angeles,40.355109,-3.700105,0.653384,0.593526,0.659865,0.319398,0.181525,0.403906,0.371881,0.290615,0.346811,0.474181,0.395173,0.521730,0.527807,0.531661,0.534406
119,Villaverde,Butarque,40.339934,-3.673388,0.431707,0.492596,0.732589,0.546429,0.201517,0.431778,0.403989,0.395061,0.352619,0.486182,0.376184,0.516746,0.589652,0.511604,0.488937
120,Villaverde,Rosales,40.355841,-3.688399,0.638547,0.683673,0.580221,0.487630,0.272962,0.367685,0.366481,0.307494,0.383974,0.433574,0.406438,0.495142,0.622091,0.487501,0.623572
121,Villaverde,San Andres,40.345539,-3.710970,0.651070,0.588445,0.813416,0.210844,0.201760,0.329589,0.360468,0.315072,0.379194,0.418344,0.420434,0.500538,0.564496,0.523189,0.600530


The following HeatMap with time shows graphically this revaluation per year and per Neighborhood

In [31]:
from folium.plugins import HeatMapWithTime
map_madrid3 = folium.Map(location=madrid_center_coords, zoom_start=11)
folium.GeoJson(madrid_neighborhoods, name='geojson').add_to(map_madrid3)
auxlist=[]
for index in range(4,df_mapa_inc_years.shape[1]):
    auxlist.append(df_mapa_inc_years.iloc[:,[2,3,index]].values.tolist())
HeatMapWithTime(auxlist,auto_play=True, radius=20).add_to(map_madrid3)
for lat, lng, District, Neighborhood in zip(df_mapa_inc_years['Latitude'], df_mapa_inc_years['Longitude'], df_mapa_inc_years['District'], df_mapa_inc_years['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, District)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(map_madrid3)
map_madrid3.save (r'inc_prices_heatmapwithtime.html')
map_madrid3

A BoxPlot will be a better analytical plot to show this results

In [32]:
import matplotlib as mpl
import matplotlib.pyplot as plt

years = list(map(str, range(2001, 2019)))
precios_distrito=precios_df.groupby('District',as_index=False).mean().drop(['Longitude','pct_inc_year'], axis=1).reset_index(drop=True)
auxlist_data=list()
auxlist_names=list()
for index, row in precios_distrito.iterrows():
    auxlist_data.append(row.drop('District').transpose())
    auxlist_names.append(row['District'])
   
fig = plt.figure(1, figsize=(16,8))
ax = fig.add_subplot(111)
ax.boxplot(auxlist_data, labels=auxlist_names)
plt.hlines(precios_distrito['2018'].mean(), xmin=0, xmax=22, colors='blue', linestyles='dashed', label='Mean Value')
ax.set_xticklabels(auxlist_names, rotation='vertical')
plt.title('Box plot of prices per square meter in each District 2001 - 2018')
plt.ylabel('Price per square meter (€)')
plt.tight_layout()
plt.gcf().savefig("Boxplot.png", format='png') 
plt.show(fig)

<Figure size 1600x800 with 1 Axes>

And the mean price of Madrid city in the last year is:

In [33]:
print('{:6.2f}'.format(precios_distrito['2018'].mean())+"€")

3437.57€


Now it's time to use Foursquare API to count the Subway and Train stations and Nurseries in each Neighborhood

In [34]:
# @hidden_cell
CLIENT_ID = "N0CZGIUOLRVFMLQIVMXN0OMNCUFRG5IITWRCA5ZGO3QOWMED"
CLIENT_SECRET = "BYWC022ZKFN3NG0M1VXOL0350YDYMDBQIRYDBLZ35S5LSZMI"
VERSION = '20180604'
LIMIT = 30

A radius of 500m from the centers will fit almost all Neighborhoods

In [35]:
radius=500
LIMIT=10

In [36]:
def searchNearbyVenues(names, category, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&intent={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            'browse',
            radius, 
            LIMIT,
            category)
        
        # make the GET request
        results = requests.get(url).json()["response"]["venues"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In case Foursquare Quota is exceeded, the data is saved in a CSV file in order to be processed and continue our work.

In [37]:
metro_station_cat="4bf58dd8d48988d1fd931735"
train_station_cat="4bf58dd8d48988d129951735"
nursery_cat="5744ccdfe4b0c0459246b4c7"
try:
    madrid_metros = searchNearbyVenues(names=precios_df['Neighborhood'],
                                        category=metro_station_cat,
                                        latitudes=precios_df['Latitude'],
                                        longitudes=precios_df['Longitude']
                                        )
    with open('madrid_metros.csv', 'w') as csv_file:
        madrid_metros.to_csv(path_or_buf=csv_file,index=False)
except KeyError:
    print("Foursquare quota exceeded, using stored data.")
    madrid_metros = pd.read_csv('madrid_metros.csv', encoding='ansi')

This is the resulting Subways table found for each Neighborhood. It is needed to be grouped and counted.

In [38]:
#madrid_metros=madrid_metros[madrid_metros['Venue Category'] == 'Metro Station']
madrid_metros.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acacias,40.404075,-3.705957,Metro Acacias,40.403787,-3.706529,Metro Station
1,Acacias,40.404075,-3.705957,Metro Embajadores,40.404755,-3.702469,Metro Station
2,Atocha,40.400687,-3.682391,Metro Menéndez Pelayo,40.404463,-3.680806,Metro Station
3,Atocha,40.400687,-3.682391,L1 Menendez Pelayo,40.404754,-3.681592,Metro Station
4,Atocha,40.400687,-3.682391,Calle del comercio,40.402995,-3.681692,Road


Let's do the same with Trains and Nurseries venues.

In [39]:
try:
    madrid_trains = searchNearbyVenues(names=precios_df['Neighborhood'],
                                        category=train_station_cat,
                                        latitudes=precios_df['Latitude'],
                                        longitudes=precios_df['Longitude']
                                        )
    with open('madrid_trains.csv', 'w') as csv_file:
                madrid_trains.to_csv(path_or_buf=csv_file,index=False)
except KeyError:
    print("Foursquare quota exceeded, using stored data.")
    madrid_trains = pd.read_csv('madrid_trains.csv', encoding='ansi')

In [40]:
#madrid_trains=madrid_trains[madrid_trains['Venue Category'] == 'Train Station']
madrid_trains.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acacias,40.404075,-3.705957,Cercanías Pirámides,40.402486,-3.711202,Train Station
1,Acacias,40.404075,-3.705957,Cercanías Embajadores,40.404780,-3.702556,Train Station
2,Atocha,40.400687,-3.682391,Via 2,40.403247,-3.686543,Platform
3,Atocha,40.400687,-3.682391,Anden 10,40.401839,-3.683745,Platform
4,Atocha,40.400687,-3.682391,C2 Atocha-Azuqueca de Henares,40.398584,-3.684508,Train


In [41]:
try:
    madrid_nurseries = searchNearbyVenues(names=precios_df['Neighborhood'],
                                        category=nursery_cat,
                                        latitudes=precios_df['Latitude'],
                                        longitudes=precios_df['Longitude']
                                        )
    with open('madrid_nurseries.csv', 'w') as csv_file:
        madrid_nurseries.to_csv(path_or_buf=csv_file,index=False)
except KeyError:
    print("Foursquare quota exceeded, using stored data.")
    madrid_nurseries = pd.read_csv('madrid_nurseries.csv', encoding='ansi')

Now, group and count venues in each Dataframe.

In [42]:
madrid_metros=madrid_metros.groupby("Neighbourhood", sort=False).count()["Venue"]
madrid_trains=madrid_trains.groupby("Neighbourhood", sort=False).count()["Venue"]
madrid_nurseries=madrid_nurseries.groupby("Neighbourhood", sort=False).count()["Venue"]

In [43]:
madrid_venues = pd.DataFrame({"Metros": madrid_metros, "Trains": madrid_trains, "Nurseries": madrid_nurseries})
madrid_venues.index.name='Neighborhood'
madrid_venues=madrid_venues.reindex(index=precios_df["Neighborhood"]).reset_index()

Resulting data is concatenated to our main Dataframe

In [44]:
precios_df=pd.concat([precios_df,madrid_venues[["Metros", "Trains", "Nurseries"]]], axis=1, sort=False)

Now we have all required Data. It's time to cluster and classify neighborhoods by similarity. K-means clustering method is chosen, so let's prepare a new suitable dataframe with relevant features.

In [45]:
precios_clustering=precios_df[['2018','pct_inc_year','Metros','Trains','Nurseries']]

In [46]:
precios_clustering=precios_clustering.fillna(0)

In [47]:
precios_clustering

,2018,pct_inc_year,Metros,Trains,Nurseries
0,4046.0,4.121014,2.0,2.0,0.0
1,2312.0,0.000000,3.0,7.0,0.0
2,3727.0,4.765614,0.0,0.0,1.0
3,3777.0,3.703018,4.0,7.0,0.0
4,3896.0,3.863440,0.0,0.0,0.0
...,...,...,...,...,...
118,1648.0,1.523792,0.0,0.0,0.0
119,1803.0,3.880243,0.0,1.0,0.0
120,1761.0,2.200722,0.0,1.0,0.0
121,1644.0,1.606152,1.0,5.0,0.0


## Clustering Neighborhoods

In [48]:
from sklearn.cluster import KMeans
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(precios_clustering)
# add clustering labels
kmeans.labels_.shape

(123,)

Once the classification is done in 5 categories, Neighborhoods are labeled according to the results

In [49]:
precios_merged = precios_df.copy()
precios_merged.insert(0, 'Cluster_Labels', kmeans.labels_.astype(int))

This is the new aspect of our data

In [50]:
precios_merged.head()

,Cluster_Labels,District,Neighborhood,2001,2002,2003,2004,2005,2006,2007,...,2015,2016,2017,2018,Latitude,Longitude,pct_inc_year,Metros,Trains,Nurseries
0,0,Arganzuela,Acacias,2237.0,2993.0,3506.0,3793.0,4178.0,4453.0,4309.0,...,2966.0,3193.0,3724.0,4046.0,40.404075,-3.705957,4.121014,2.0,2.0,NaN
1,1,Arganzuela,Atocha,2312.0,2312.0,2312.0,2312.0,2312.0,2312.0,2312.0,...,2312.0,2312.0,2312.0,2312.0,40.400687,-3.682391,0.000000,3.0,7.0,NaN
2,0,Arganzuela,Chopera,1939.0,2576.0,3222.0,3434.0,3680.0,3972.0,3942.0,...,2355.0,2649.0,3340.0,3727.0,40.394893,-3.699705,4.765614,NaN,NaN,1.0
3,0,Arganzuela,Delicias,2254.0,3008.0,3279.0,3751.0,3882.0,4295.0,4138.0,...,2592.0,3025.0,3566.0,3777.0,40.397292,-3.689495,3.703018,4.0,7.0,NaN
4,0,Arganzuela,Imperial,2264.0,2934.0,3591.0,3507.0,3711.0,4254.0,4240.0,...,2697.0,2920.0,3382.0,3896.0,40.406915,-3.717329,3.863440,NaN,NaN,NaN


It's time to plot this labels into our Neighborhoods map, we will use 5 diferent colors to show each label. Also, we will show additional data to visualize the characteristics of each Cluster. Click on each point to explore them

In [51]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=madrid_center_coords, zoom_start=11)
folium.GeoJson(madrid_neighborhoods, name='geojson').add_to(map_clusters)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(precios_merged['Latitude'], precios_merged['Longitude'], precios_merged['Neighborhood'], precios_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save(r"map_clusters.html")
map_clusters

A more analytical method to view the characteristics of each Cluster is to show the mean values of each one

In [52]:
precios_merged.Latitude=precios_merged.Latitude.astype(np.float64)
precios_merged.groupby("Cluster_Labels").mean()[['2018','pct_inc_year','Metros','Trains','Nurseries']]

,2018,pct_inc_year,Metros,Trains,Nurseries
Cluster_Labels,,,,,
0,4155.967742,3.614854,2.111111,3.428571,1.285714
1,2046.263158,1.979740,1.521739,2.000000,1.111111
2,5096.928571,4.286719,4.000000,3.833333,1.200000
3,3272.028571,4.457942,1.304348,1.200000,1.166667
4,6666.400000,4.594247,4.000000,3.000000,NaN


## Results

With this results, and the requirements of ARPS, Cluster 1 is the best candidate. Let's drow these Neighborhoods by selecting the polygons in the GeoJson.

In [53]:
map_cluster1 = folium.Map(location=madrid_center_coords, zoom_start=11)
cluster1_data=precios_merged[precios_merged.Cluster_Labels == 1]
output_json=madrid_neighborhoods.copy()

output_json['features'] = [neigh for neigh in madrid_neighborhoods['features'] for index, row in cluster1_data.iterrows()
                          if Point((row['Longitude'], row['Latitude'])).within(Polygon(neigh['geometry']['coordinates'][0][0]))]

folium.GeoJson(output_json, name='geojson').add_to(map_cluster1)

for lat, lng, District, Neighborhood, Price, Nurseries, Metros, Trains in zip(cluster1_data['Latitude'], cluster1_data['Longitude'], cluster1_data['District'], cluster1_data['Neighborhood'], cluster1_data['2018'], cluster1_data['Nurseries'], cluster1_data['Metros'], cluster1_data['Trains']):
    label = '{}, {}, {}€/m^2, Nurseries:{},Metros:{},Trains:{}'.format(Neighborhood, District, Price,Nurseries,Metros,Trains)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(map_cluster1)

map_cluster1.save("map_cluster1.html")
map_cluster1

As some of this candidates have no Subway or Train stations, a new filter is applied. We will chose only Neighborhoods containing at least 1 Metro station, and 1 Train Station

In [54]:
cluster1_data_filtered=precios_merged.loc[(precios_merged["Cluster_Labels"] == 1) & (precios_merged["Metros"]>0) & (precios_merged["Trains"]>0)]
map_cluster1_filtered = folium.Map(location=madrid_center_coords, zoom_start=11)
output_json=madrid_neighborhoods.copy()

output_json['features'] = [neigh for neigh in madrid_neighborhoods['features'] for index, row in cluster1_data_filtered.iterrows()
                          if Point((row['Longitude'], row['Latitude'])).within(Polygon(neigh['geometry']['coordinates'][0][0]))]

folium.GeoJson(output_json, name='geojson').add_to(map_cluster1_filtered)

for lat, lng, District, Neighborhood, Price, Nurseries, Metros, Trains in zip(cluster1_data_filtered['Latitude'], cluster1_data_filtered['Longitude'], cluster1_data_filtered['District'], cluster1_data_filtered['Neighborhood'], cluster1_data_filtered['2018'], cluster1_data_filtered['Nurseries'], cluster1_data_filtered['Metros'], cluster1_data_filtered['Trains']):
    label = '{}, {}, {}€/m^2, Nurseries:{},Metros:{},Trains:{}'.format(Neighborhood, District, Price,Nurseries,Metros,Trains)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(map_cluster1_filtered)
    
map_cluster1_filtered.save("map_cluster1_filtered.html")
map_cluster1_filtered

The resulting Top 5 Candidates are:

In [55]:
cluster1_data_filtered.fillna(0)[['District','Neighborhood', 'pct_inc_year', 'Metros', 'Trains', 'Nurseries']]

,District,Neighborhood,pct_inc_year,Metros,Trains,Nurseries
1,Arganzuela,Atocha,0.000000,3.0,7.0,0.0
41,Ciudad Lineal,Ventas,2.457383,2.0,1.0,0.0
59,Latina,Las Aguilas,1.536636,1.0,1.0,1.0
107,Usera,Almendrales,1.827758,2.0,1.0,1.0
121,Villaverde,San Andres,1.606152,1.0,5.0,0.0
